# Multidimensional Coordinates example using CMIP6 Pangeo ocean model data

## Import python packages

In [ ]:
import xarray as xr

xr.set_options(display_style="html")
import intake
import cftime
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cartopy.crs as ccrs
import numpy as np

%matplotlib inline

## Open CMIP6 online catalog

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

## Search corresponding data 

In [ ]:
cat = col.search(
    source_id=["MPI-ESM1-2-HR"],
    experiment_id=["historical"],
    table_id=["Omon"],
    variable_id=["chl"],
    member_id=["r9i1p1f1"],
)
cat.df

## Create dictionary from the list of datasets we found
- This step may take several minutes so be patient!

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={"use_cftime": True})

In [ ]:
list(dset_dict.keys())

## Open dataset

- Use `xarray` python package to analyze netCDF dataset
- `open_dataset` allows to get all the metadata without loading data into memory. 
- with `xarray`, we only load into memory what is needed.

In [ ]:
dset = dset_dict["CMIP.MPI-M.MPI-ESM1-2-HR.historical.Omon.gn"]
dset

### Get metadata corresponding to Chl

In [ ]:
print(dset["chl"])

In [ ]:
dset.time.values

# Visualization

## shift longitude from 0,360 to -180,180

In [ ]:
dset.coords["longitude"] = (dset["longitude"] + 180) % 360 - 180

In [ ]:
dset["chl"].latitude.values.min(), dset["chl"].latitude.values.max()

In [ ]:
dset["chl"].longitude.values.min(), dset["chl"].longitude.values.max()

## Select geographical area, time and level

In [ ]:
dset_selection = (
    dset.isel(time=0, lev=0)
    .where((dset.latitude > 50) & (dset.longitude > -30) & (dset.longitude < 30))
    .squeeze()
)

In [ ]:
def polarCentral_set_latlim(lat_lims, ax):
    ax.set_extent([-180, 180, lat_lims[0], lat_lims[1]], ccrs.PlateCarree())
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)

In [ ]:
fig = plt.figure(1, figsize=[10, 10])

# Fix extent
minval = 0
maxval = 100.0

ax = plt.subplot(1, 1, 1, projection=ccrs.NorthPolarStereo())
ax.coastlines()
ax.gridlines()
polarCentral_set_latlim([50, 90], ax)
dset_selection.chl.plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=False,
    cmap="coolwarm",
)